# Rapport du Projet Python

auteur :

- Guillaume Dorschner
- Louis Anne
- Valentin Grateau

# Getting Started

## Prérequis

Au démarrage il faut installer les dépendances du projet. Pour cela il faut exécuter la commande suivante:

```bash
pip install -r requirements.txt
```

## Exécution du code

Pour lancer le server, il faut exécuter le fichier `mange.py` avec la commande suivante:

```bash
python manage.py runserver
```

## Ratio contribution

Guillaume : Django 75% |  Traitement des données 0% | Analyse des données/Graphes  15% | Rapport 30%

Louis : Django 25% |  Traitement des données  10% | Analyse des données/Graphes 20% | Rapport 50%

Valentin : Django  0% |  Traitement des données  90% | Analyse des données/Graphes  65% | Rapport 20%


## Avancement et difficultés

### Traitement des données

Lors de la lecture des données, la première chose à faire est de convertir aux bons types les valeurs donc la colonne « Date mutation » en DateTime ainsi que toutes les autres valeurs en float64. Ensuite on passe à l’analyse et au nettoyage. On utilise donc set_option('display.max_columns') afin de pouvoir observer toutes les colonnes en mêmes temps. On regarde ensuite quelles valeurs sont grandement manquante afin de les supprimer. Une fois les colonnes sélectionnées selon nos critères qui étaient le nombre de valeurs nulles, si la même valeur se répétait ou encore si nous ne comptions pas apporter d’importance à cette variable dans nos analyses. Notre DataFrame comporte donc 24 colonnes au lieu de 43 au départ. Enfin, nous nous sommes aperçus que des valeurs complètement incohérentes dans le département de Paris existaient et faussaient notre calcul du prix moyen du mètre carré. Nous avons donc supprimé de nos données celles qui dépassaient 100 000 000 euros de valeur foncière et qui n’avaient pas de « Type local ». Nous exportons donc au format csv ces DataFrame traitées afin de faire nos analyses. 

### Analyse des données/Graphes

Concernant les analyses, nous avons décidé de faire des graphes assez diversifiés afin que la personne cherchant à en tirer une analyse puisse avoir plusieurs visualisations possibles et que cela lui permette donc de mieux observer ce qu’elle souhaite. La valeur la plus courante étant le prix du mètre carré, au début nous avions commis l’erreur de choix de calculer cette valeur avec la surface réelle bâti au lieu de la surface Carrez. Nous avons donc ensuite calculé cette valeur avec cette surface Carrez quand cela était possible. Cependant, certaines valeurs restent beaucoup trop grandes et c’est donc pour cela que nous avons décidé de supprimer dans certains graphes les valeurs du prix du mètre carré au-delà de 25 000 euros. Nous avons ensuite réalisé en utilisant ou non cette méthode afin de voir la différence de résultat que nous pouvions avoir sur les graphes représentant cette valeur du prix du mètre carré. Certains graphes ont des valeurs que nous savons fausses et c'est volontaire de les avoir laissé afin de montrer l'évolution de notre analyse des données au fur et à mesure des graphes que nous avons réalisés. Comme par exemple avec les deux cartes représentant le prix du mètre carré par départements. La première possède des valeurs faites avec la surface réelle bati et les valeurs trop grandes n'ont pas été retirées. Tandis que dans le second graphes, les valeurs sont calculées avec la surface Carrez si possible sinon avec la surface réelle bati et les valeurs beaucoup trop grandes pour être correctes ont été supprimées afin d'avoir une réelle observation. 

### Réalisation du site en Django 

Au démarrage, le serveur charge toutes les données nécessaires en mémoire. Cela peut prendre un certain temps en fonction de la taille des données. C'est une étape essentielle pour garantir des temps de réponse rapides lorsque les utilisateurs demandent des graphiques ou des analyses.
Ensuite, nous avons qui représente le REST du server. Chaque vue est responsable de traiter une demande HTTP. Les fonctions index / about / analyse_intra / analyse_inter affiche les pages / template html. Mais y il a aussi get_graph qui permet d’afficher les graphs. Pour rendre les interactions utilisateur plus fluides et éviter que le client attend une pages longtemps, le serveur envoye la pages, puis recuppe les graphique petit à petit avec Ajax. Cette approche permet à l'utilisateur de continuer à interagir avec la page pendant que les graphiques sont chargés, évitant ainsi une attente prolongée et améliorant l'expérience utilisateur globale.
Nous utilisons a bibliothèque mpld3 pour convertir les graphiques Matplotlib en HTML.

# Traitement des données

## Import

#Plotly
#!pip install plotly 

In [2]:
import pandas as pd
import numpy as np
import re
import datetime
import folium
import random
import json
from datetime import timedelta
from urllib.request import urlopen
import numpy as np
import pandas as pd
from plotly.offline import plot, iplot, init_notebook_mode
init_notebook_mode(connected=True)
import warnings
warnings.filterwarnings('ignore')

## Extraction of data

In [4]:
annee = "2018"
df = pd.read_csv(f'./data/valeursfoncieres-{annee}.txt', sep = '|',decimal = ',')
df["Date mutation"] = pd.to_datetime(df["Date mutation"], format="%d/%m/%Y")
display(df)

,Identifiant de document,Reference document,1 Articles CGI,2 Articles CGI,3 Articles CGI,4 Articles CGI,5 Articles CGI,No disposition,Date mutation,Nature mutation,...,Surface Carrez du 5eme lot,Nombre de lots,Code type local,Type local,Identifiant local,Surface reelle bati,Nombre pieces principales,Nature culture,Nature culture speciale,Surface terrain
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,2018-01-03,Vente,...,NaN,2,2.0,Appartement,NaN,73.0,4.0,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,2018-01-03,Vente,...,NaN,1,3.0,Dépendance,NaN,0.0,0.0,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,2018-01-04,Vente,...,NaN,0,1.0,Maison,NaN,163.0,4.0,S,NaN,949.0
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,2018-01-04,Vente,...,NaN,0,1.0,Maison,NaN,163.0,4.0,AG,JARD,420.0
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,2018-01-04,Vente,...,NaN,0,1.0,Maison,NaN,51.0,2.0,AG,JARD,420.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3335287,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,2018-12-28,Vente,...,NaN,2,2.0,Appartement,NaN,150.0,4.0,NaN,NaN,NaN
3335288,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,2018-12-03,Vente,...,NaN,1,2.0,Appartement,NaN,34.0,1.0,NaN,NaN,NaN
3335289,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,2018-12-28,Vente,...,NaN,1,4.0,Local industriel. commercial ou assimilé,NaN,60.0,0.0,NaN,NaN,NaN
3335290,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,2018-12-13,Adjudication,...,NaN,2,2.0,Appartement,NaN,54.0,2.0,NaN,NaN,NaN


In [5]:
data = df.copy()

## Creation of a DataFrame

In [6]:
pd.set_option('display.max_columns', None)
display(data)

,Identifiant de document,Reference document,1 Articles CGI,2 Articles CGI,3 Articles CGI,4 Articles CGI,5 Articles CGI,No disposition,Date mutation,Nature mutation,Valeur fonciere,No voie,B/T/Q,Type de voie,Code voie,Voie,Code postal,Commune,Code departement,Code commune,Prefixe de section,Section,No plan,No Volume,1er lot,Surface Carrez du 1er lot,2eme lot,Surface Carrez du 2eme lot,3eme lot,Surface Carrez du 3eme lot,4eme lot,Surface Carrez du 4eme lot,5eme lot,Surface Carrez du 5eme lot,Nombre de lots,Code type local,Type local,Identifiant local,Surface reelle bati,Nombre pieces principales,Nature culture,Nature culture speciale,Surface terrain
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,2018-01-03,Vente,109000.0,13.0,NaN,RUE,1660,GEN LOGEROT,1000.0,BOURG-EN-BRESSE,1,53,NaN,AN,73,NaN,1,NaN,31.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,2.0,Appartement,NaN,73.0,4.0,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,2018-01-03,Vente,109000.0,13.0,NaN,RUE,1660,GEN LOGEROT,1000.0,BOURG-EN-BRESSE,1,53,NaN,AN,73,NaN,13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,3.0,Dépendance,NaN,0.0,0.0,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,2018-01-04,Vente,239300.0,4.0,NaN,RUE,0025,DE LA BARMETTE,1250.0,NIVIGNE ET SURAN,1,95,NaN,AH,186,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1.0,Maison,NaN,163.0,4.0,S,NaN,949.0
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,2018-01-04,Vente,239300.0,4.0,NaN,RUE,0025,DE LA BARMETTE,1250.0,NIVIGNE ET SURAN,1,95,NaN,AH,186,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1.0,Maison,NaN,163.0,4.0,AG,JARD,420.0
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,2018-01-04,Vente,239300.0,4.0,NaN,RUE,0025,DE LA BARMETTE,1250.0,NIVIGNE ET SURAN,1,95,NaN,AH,186,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1.0,Maison,NaN,51.0,2.0,AG,JARD,420.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3335287,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,2018-12-28,Vente,1192307.0,1.0,NaN,PAS,2012,CHOISEUL,75002.0,PARIS 02,75,102,NaN,AD,118,NaN,43.0,120.54,44.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,2.0,Appartement,NaN,150.0,4.0,NaN,NaN,NaN
3335288,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,2018-12-03,Vente,383000.0,12.0,NaN,RUE,7561,POISSONNIERE,75002.0,PARIS 02,75,102,NaN,AO,85,NaN,9.0,34.78,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,2.0,Appartement,NaN,34.0,1.0,NaN,NaN,NaN
3335289,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,2018-12-28,Vente,746000.0,18.0,NaN,RUE,8548,STE CROIX BRETONNERIE,75004.0,PARIS 04,75,104,NaN,AH,53,NaN,8.0,63.46,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,4.0,Local industriel. commercial ou assimilé,NaN,60.0,0.0,NaN,NaN,NaN
3335290,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,2018-12-13,Adjudication,645000.0,9.0,NaN,BD,6559,MORLAND,75004.0,PARIS 04,75,104,NaN,AS,74,NaN,101.0,NaN,122.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,2.0,Appartement,NaN,54.0,2.0,NaN,NaN,NaN


## Cleaning of data

In [7]:
columns = data.columns
drop_columns =  [columns[0],columns[1],columns[2],columns[3],columns[4],columns[5],columns[6],columns[7],columns[12],columns[17],columns[20],columns[23],columns[26],columns[28],columns[30],columns[32],columns[37],columns[41]]
data.drop(columns = drop_columns, inplace=True, axis=1)
display(data)

print(drop_columns)

,Date mutation,Nature mutation,Valeur fonciere,No voie,Type de voie,Code voie,Voie,Code postal,Code departement,Code commune,Section,No plan,1er lot,Surface Carrez du 1er lot,Surface Carrez du 2eme lot,Surface Carrez du 3eme lot,Surface Carrez du 4eme lot,Surface Carrez du 5eme lot,Nombre de lots,Code type local,Type local,Surface reelle bati,Nombre pieces principales,Nature culture,Surface terrain
0,2018-01-03,Vente,109000.0,13.0,RUE,1660,GEN LOGEROT,1000.0,1,53,AN,73,1,NaN,NaN,NaN,NaN,NaN,2,2.0,Appartement,73.0,4.0,NaN,NaN
1,2018-01-03,Vente,109000.0,13.0,RUE,1660,GEN LOGEROT,1000.0,1,53,AN,73,13,NaN,NaN,NaN,NaN,NaN,1,3.0,Dépendance,0.0,0.0,NaN,NaN
2,2018-01-04,Vente,239300.0,4.0,RUE,0025,DE LA BARMETTE,1250.0,1,95,AH,186,NaN,NaN,NaN,NaN,NaN,NaN,0,1.0,Maison,163.0,4.0,S,949.0
3,2018-01-04,Vente,239300.0,4.0,RUE,0025,DE LA BARMETTE,1250.0,1,95,AH,186,NaN,NaN,NaN,NaN,NaN,NaN,0,1.0,Maison,163.0,4.0,AG,420.0
4,2018-01-04,Vente,239300.0,4.0,RUE,0025,DE LA BARMETTE,1250.0,1,95,AH,186,NaN,NaN,NaN,NaN,NaN,NaN,0,1.0,Maison,51.0,2.0,AG,420.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3335287,2018-12-28,Vente,1192307.0,1.0,PAS,2012,CHOISEUL,75002.0,75,102,AD,118,43.0,120.54,NaN,NaN,NaN,NaN,2,2.0,Appartement,150.0,4.0,NaN,NaN
3335288,2018-12-03,Vente,383000.0,12.0,RUE,7561,POISSONNIERE,75002.0,75,102,AO,85,9.0,34.78,NaN,NaN,NaN,NaN,1,2.0,Appartement,34.0,1.0,NaN,NaN
3335289,2018-12-28,Vente,746000.0,18.0,RUE,8548,STE CROIX BRETONNERIE,75004.0,75,104,AH,53,8.0,63.46,NaN,NaN,NaN,NaN,1,4.0,Local industriel. commercial ou assimilé,60.0,0.0,NaN,NaN
3335290,2018-12-13,Adjudication,645000.0,9.0,BD,6559,MORLAND,75004.0,75,104,AS,74,101.0,NaN,NaN,NaN,NaN,NaN,2,2.0,Appartement,54.0,2.0,NaN,NaN


['Identifiant de document', 'Reference document', '1 Articles CGI', '2 Articles CGI', '3 Articles CGI', '4 Articles CGI', '5 Articles CGI', 'No disposition', 'B/T/Q', 'Commune', 'Prefixe de section', 'No Volume', '2eme lot', '3eme lot', '4eme lot', '5eme lot', 'Identifiant local', 'Nature culture speciale']


In [ ]:
display(data)

?????

In [8]:
paris = data[(data["Code departement"] == 75) & (data["Type local"] != "Dépendance") & (data["Type local"] != "Local industriel. commercial ou assimilé")].reset_index(drop = True)
paris["Prix mètre carré"] = paris["Valeur fonciere"]/paris["Surface reelle bati"]
display(paris)

,Date mutation,Nature mutation,Valeur fonciere,No voie,Type de voie,Code voie,Voie,Code postal,Code departement,Code commune,Section,No plan,1er lot,Surface Carrez du 1er lot,Surface Carrez du 2eme lot,Surface Carrez du 3eme lot,Surface Carrez du 4eme lot,Surface Carrez du 5eme lot,Nombre de lots,Code type local,Type local,Surface reelle bati,Nombre pieces principales,Nature culture,Surface terrain,Prix mètre carré
0,2018-01-04,Vente,1202000.0,10.0,AV,7250,PERCIER,75008.0,75,108,CJ,3,77.0,159.20,NaN,NaN,NaN,NaN,3,2.0,Appartement,120.0,2.0,NaN,NaN,10016.666667
1,2018-01-09,Vente,130000.0,20.0,RUE,8520,SAINT CLAUDE,75003.0,75,103,AL,72,206.0,NaN,NaN,NaN,NaN,NaN,4,2.0,Appartement,22.0,1.0,NaN,NaN,5909.090909
2,2018-01-03,Vente,15000.0,12.0,RUE,9905,VOLTA,75003.0,75,103,AE,136,7.0,8.30,NaN,NaN,NaN,NaN,1,2.0,Appartement,9.0,1.0,NaN,NaN,1666.666667
3,2018-01-03,Vente,650000.0,174.0,BD,4485,HAUSSMANN,75008.0,75,108,BD,36,59.0,96.58,NaN,NaN,NaN,NaN,2,2.0,Appartement,96.0,3.0,NaN,NaN,6770.833333
4,2018-01-05,Vente,3000.0,187.0,RUE,3518,DU FAUBOURG SAINT HONORE,75008.0,75,108,AW,27,37.0,NaN,NaN,NaN,NaN,NaN,1,2.0,Appartement,58.0,3.0,NaN,NaN,51.724138
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44027,2018-12-26,Vente,220000.0,14.0,RUE,5702,DES LIONS SAINT PAUL,75004.0,75,104,AQ,127,126.0,NaN,NaN,NaN,NaN,NaN,2,2.0,Appartement,29.0,1.0,NaN,NaN,7586.206897
44028,2018-12-28,Vente,1192307.0,1.0,PAS,2012,CHOISEUL,75002.0,75,102,AD,118,43.0,120.54,NaN,NaN,NaN,NaN,2,2.0,Appartement,150.0,4.0,NaN,NaN,7948.713333
44029,2018-12-03,Vente,383000.0,12.0,RUE,7561,POISSONNIERE,75002.0,75,102,AO,85,9.0,34.78,NaN,NaN,NaN,NaN,1,2.0,Appartement,34.0,1.0,NaN,NaN,11264.705882
44030,2018-12-13,Adjudication,645000.0,9.0,BD,6559,MORLAND,75004.0,75,104,AS,74,101.0,NaN,NaN,NaN,NaN,NaN,2,2.0,Appartement,54.0,2.0,NaN,NaN,11944.444444


In [9]:
erreur = data[(data["Valeur fonciere"] <= 100000000)&(data["Type local"].notnull())]
paris = erreur[(erreur["Code departement"] == 75)]
display(paris.sort_values('Valeur fonciere').tail(50))

,Date mutation,Nature mutation,Valeur fonciere,No voie,Type de voie,Code voie,Voie,Code postal,Code departement,Code commune,Section,No plan,1er lot,Surface Carrez du 1er lot,Surface Carrez du 2eme lot,Surface Carrez du 3eme lot,Surface Carrez du 4eme lot,Surface Carrez du 5eme lot,Nombre de lots,Code type local,Type local,Surface reelle bati,Nombre pieces principales,Nature culture,Surface terrain
3317614,2018-12-21,Vente,91600000.0,167.0,BD,5951,MALESHERBES,75017.0,75,117,BS,25,NaN,NaN,NaN,NaN,NaN,NaN,0,2.0,Appartement,185.0,7.0,S,2763.0
3317613,2018-12-21,Vente,91600000.0,167.0,BD,5951,MALESHERBES,75017.0,75,117,BS,25,NaN,NaN,NaN,NaN,NaN,NaN,0,2.0,Appartement,140.0,5.0,S,2763.0
3317612,2018-12-21,Vente,91600000.0,167.0,BD,5951,MALESHERBES,75017.0,75,117,BS,25,NaN,NaN,NaN,NaN,NaN,NaN,0,2.0,Appartement,213.0,7.0,S,2763.0
3317611,2018-12-21,Vente,91600000.0,167.0,BD,5951,MALESHERBES,75017.0,75,117,BS,25,NaN,NaN,NaN,NaN,NaN,NaN,0,2.0,Appartement,232.0,7.0,S,2763.0
3317610,2018-12-21,Vente,91600000.0,167.0,BD,5951,MALESHERBES,75017.0,75,117,BS,25,NaN,NaN,NaN,NaN,NaN,NaN,0,2.0,Appartement,226.0,8.0,S,2763.0
3317609,2018-12-21,Vente,91600000.0,167.0,BD,5951,MALESHERBES,75017.0,75,117,BS,25,NaN,NaN,NaN,NaN,NaN,NaN,0,2.0,Appartement,223.0,7.0,S,2763.0
3317608,2018-12-21,Vente,91600000.0,167.0,BD,5951,MALESHERBES,75017.0,75,117,BS,25,NaN,NaN,NaN,NaN,NaN,NaN,0,2.0,Appartement,103.0,3.0,S,2763.0
3317639,2018-12-21,Vente,91600000.0,167.0,BD,5951,MALESHERBES,75017.0,75,117,BS,25,NaN,NaN,NaN,NaN,NaN,NaN,0,3.0,Dépendance,0.0,0.0,S,2763.0
3317638,2018-12-21,Vente,91600000.0,167.0,BD,5951,MALESHERBES,75017.0,75,117,BS,25,NaN,NaN,NaN,NaN,NaN,NaN,0,3.0,Dépendance,0.0,0.0,S,2763.0
3317641,2018-12-21,Vente,91600000.0,167.0,BD,5951,MALESHERBES,75017.0,75,117,BS,25,NaN,NaN,NaN,NaN,NaN,NaN,0,3.0,Dépendance,0.0,0.0,S,2763.0


## Create Region column

In [13]:
with open('./data/regions/regions_dict.json', 'r') as f:
    regions_dict = json.load(f)

with open('./data/departements/departements_dict.json', 'r') as f:
    departments_dict = json.load(f)

erreur['Code departement'] = erreur['Code departement'].apply(lambda x: str(x).zfill(2))

regions_dict = {dep: region for region, deps in regions_dict.items() for dep in deps}
departments_dict = {k: v for k, v in departments_dict.items()}

erreur['Region'] = erreur['Code departement'].map(regions_dict)

Saving the DataFrame in a csv file (to use it in Django), we did it for each year

In [15]:
erreur.to_csv(f'./data/annee_traitee/{annee}.csv')

# Analyse intra data 2022

Importation des librairies et des données ainsi que l'import des dapartements et des régions

In [ ]:
import matplotlib.pyplot as plt
import datetime
import folium
import pandas as pd
import numpy as np
from branca.colormap import linear
from urllib.request import urlopen
from plotly.offline import plot, iplot, init_notebook_mode
init_notebook_mode(connected=True)
import plotly.graph_objects as go
import plotly.express as px
import plotly.subplots as sp


dth = '#ff2e63' 
rec = '#21bf73'
act = '#fe9801'
cnf = '#393e46'

DEPARTMENTS = json.load(open("../data/departements/departements_dict.json"))

REGIONS = json.load(open("../data/regions/regions_dict.json"))

data = pd.read_csv(r"../data/annee_traitee/2022.csv",low_memory=False,sep=",",header=0,index_col=0)

data["Date mutation"] = pd.to_datetime(data["Date mutation"], format="%Y-%m-%d")

### Données

In [ ]:
display(data)

In [ ]:
paris = data[(data["Code departement"] == '75') & (data["Type local"] != "Dépendance")& (data["Type local"] != "Local industriel. commercial ou assimilé")].reset_index(drop = True)
paris["Prix mètre carré"] = paris["Valeur fonciere"]/paris["Surface reelle bati"]
display(paris)

## Prix au m2

In [ ]:
m2 = data[(data["Type local"] != "Dépendance")& (data["Type local"] != "Local industriel. commercial ou assimilé")].reset_index(drop = True)
print(m2['Code departement'].nunique())
m2["carrez_sum"] = m2["Surface Carrez du 1er lot"].fillna(0)  +  m2["Surface Carrez du 2eme lot"].fillna(0) + m2["Surface Carrez du 3eme lot"].fillna(0) + m2["Surface Carrez du 4eme lot"].fillna(0) + m2["Surface Carrez du 5eme lot"].fillna(0)
m2["Prix mètre carré"] = np.where(m2["carrez_sum"] != 0,m2["Valeur fonciere"]/m2["carrez_sum"],m2["Valeur fonciere"]/m2["Surface reelle bati"])
m2 = m2.drop(np.where(m2['Prix mètre carré'] > 25000)[0])
prix_m2_departement = m2.groupby('Code departement',as_index=False)['Prix mètre carré'].mean()
 
display(prix_m2_departement)

## top 3 des departements les plus cher / moins cher

In [ ]:
top5_chers = pd.DataFrame(prix_m2_departement.sort_values(by="Prix mètre carré",ascending=False).head(5))
for i in top5_chers["Code departement"]:
    top5_chers.loc[top5_chers["Code departement"] == i,"Département"] = DEPARTMENTS.get(str(i))

display(top5_chers)


top5_moins_chers = pd.DataFrame(prix_m2_departement.sort_values(by="Prix mètre carré",ascending=True).head(5))

for i in top5_moins_chers["Code departement"]:
    top5_moins_chers.loc[top5_moins_chers["Code departement"] == i,"Département"] = DEPARTMENTS.get(str(i))

display(top5_moins_chers)

In [ ]:
temp = top5_chers.drop(columns="Code departement")
temp.style.background_gradient(cmap='Reds')

In [ ]:
temp2 = top5_moins_chers.drop(columns = "Code departement")
temp2.style.background_gradient(cmap='Greens')

In [ ]:
dict_vol_ventes = data.groupby(["Code departement"])["Valeur fonciere"].sum().reset_index()
dict_vol_ventes.columns = ["Code departement","Volume monétaire"]
dict_vol_ventes["Volume monétaire"] = round(dict_vol_ventes["Volume monétaire"]/1000000000,2)
for i in dict_vol_ventes["Code departement"]:
    dict_vol_ventes.loc[dict_vol_ventes["Code departement"] == i,"Département"] = DEPARTMENTS.get(str(i))
    
dict_vol_ventes

In [ ]:
temp = dict_vol_ventes
temp = temp.sort_values('Code departement', ascending=False)

fig = px.bar(temp.sort_values('Volume monétaire', ascending=False).head(10).sort_values('Volume monétaire', ascending=True), 
             x="Volume monétaire", y="Département", text='Volume monétaire', orientation='h', 
             width=700, height=600, range_x = [0, 500], title='Volume monétaire par département en Milliards')
fig.update_traces(marker_color=act, opacity=0.8, textposition='outside')
fig.show()

In [ ]:
data["Month mutation"] = data["Date mutation"].dt.month
prix_metre_carre = data[(data["Type local"] != "Dépendance")].reset_index(drop = True)
prix_metre_carre = prix_metre_carre.dropna(subset=['Type local'])

prix_metre_carre["Prix mètre carré"] = prix_metre_carre["Valeur fonciere"]/prix_metre_carre["Surface reelle bati"]
display(prix_metre_carre)

In [ ]:
temp = data[(data["Type local"] != "Dépendance")& (data["Type local"] != "Local industriel. commercial ou assimilé")].reset_index(drop = True)

temp['Prix mètre carré Paris'] = temp[(temp["Code departement"] == '75')]["Valeur fonciere"]/temp[(temp["Code departement"] == '75')]["Surface reelle bati"]

temp['Prix mètre carré Nord'] = temp[(temp["Code departement"] == '59')]["Valeur fonciere"]/temp[(temp["Code departement"] == '59')]["Surface reelle bati"]

temp['Prix mètre carré Alpes-Maritimes'] = temp[(temp["Code departement"] == '6')]["Valeur fonciere"]/temp[(temp["Code departement"] == '6')]["Surface reelle bati"]
temp =temp.replace(np.inf, np.nan)
display(temp['Prix mètre carré Paris'].mean())
display(temp)

# temp['No. of Recovered to 1 Death Case'] = round(temp['Recovered']/temp['Deaths'], 3)
temp = temp.groupby('Month mutation')["Prix mètre carré Paris","Prix mètre carré Nord","Prix mètre carré Alpes-Maritimes"].mean().reset_index()

temp = temp.melt(id_vars='Month mutation', value_vars=['Prix mètre carré Paris','Prix mètre carré Nord','Prix mètre carré Alpes-Maritimes'], 
                 var_name='Departements', value_name='Value')

fig = px.line(temp, x="Month mutation", y="Value", color='Departements', log_y=True, 
              title='prix du mètre carré par mois en fonction de 3 départements', color_discrete_sequence=[dth, rec,act])
fig.show()



In [ ]:
temp = data[(data["Type local"] != "Dépendance")& (data["Type local"] != "Local industriel. commercial ou assimilé")].reset_index(drop = True)
carrez = data[(data["Type local"] != "Dépendance")& (data["Type local"] != "Local industriel. commercial ou assimilé")].reset_index(drop = True)
temp["carrez_sum"] = temp["Surface Carrez du 1er lot"].fillna(0)  +  temp["Surface Carrez du 2eme lot"].fillna(0) + temp["Surface Carrez du 3eme lot"].fillna(0) + temp["Surface Carrez du 4eme lot"].fillna(0) + temp["Surface Carrez du 5eme lot"].fillna(0)
temp["Prix mètre carré"] = np.where(temp["carrez_sum"] != 0,temp["Valeur fonciere"]/temp["carrez_sum"],temp["Valeur fonciere"]/temp["Surface reelle bati"])
temp = temp.drop(np.where(temp['Prix mètre carré'] > 25000)[0])

temp['Prix mètre carré Paris'] = temp[(temp["Code departement"] == '75')]["Prix mètre carré"]
temp['Prix mètre carré Nord'] = temp[(temp["Code departement"] == '59')]["Prix mètre carré"]
temp['Prix mètre carré Alpes-Maritimes'] = temp[(temp["Code departement"] == '6')]["Prix mètre carré"]

temp =temp.replace(np.inf, np.nan)
display(temp['Prix mètre carré Paris'].mean())
display(temp)

# temp['No. of Recovered to 1 Death Case'] = round(temp['Recovered']/temp['Deaths'], 3)
temp = temp.groupby('Month mutation')["Prix mètre carré Paris","Prix mètre carré Nord","Prix mètre carré Alpes-Maritimes"].mean().reset_index()

temp = temp.melt(id_vars='Month mutation', value_vars=['Prix mètre carré Paris','Prix mètre carré Nord','Prix mètre carré Alpes-Maritimes'], 
                 var_name='Departements', value_name='Value')

fig = px.line(temp, x="Month mutation", y="Value", color='Departements', log_y=True, 
              title='prix du mètre carré par mois en fonction de 3 départements', color_discrete_sequence=[dth, rec,act])
fig.show()


## répartition des types de biens

In [ ]:
type_counts = data['Type local'].value_counts()

type_counts.plot(kind='pie', autopct='%1.1f%%', figsize=(10,10),legend=True,colors=['olivedrab', 'rosybrown', 'gray', 'saddlebrown'])
plt.ylabel('')
plt.title('Répartition des types de biens')
plt.show()

On remarque que le type de bien le plus vendu sont les dépendances car plusieurs dépendances peuvent être vendu pour un appartement ou une maison.
l'immobilier d'habitation (maisons, appartements, dépendances) représentent 94.8% des bien immobiliers vendus contre 5.2% pour des achats d'immobilier commercial.
Ces données montrent une égale proportion entre les achats d'appartement et les achats de maison alors qu'en 2018, 66% des français habite dans une maison.

## heat map

In [ ]:
df = data

property_changes = df['Code departement'].value_counts().reset_index()
property_changes.columns = ['Code', 'property_changes']

property_dict = property_changes.set_index('Code')['property_changes'].to_dict()

with open('../data/departements/departements.geojson') as f:
    zone = json.load(f)

colormap = linear.YlOrRd_09.scale(
    property_changes.property_changes.min(),
    property_changes.property_changes.max())

for feature in zone['features']:
    feature['properties']['property_changes'] = property_dict.get(feature['properties']['code'], None)

def style_function(feature):
    property_changes = feature['properties']['property_changes']
    return {
        'fillOpacity': 0.7,
        'weight': 2,
        'color': 'black',
        'fillColor': '#fff' if property_changes is None else colormap(property_changes)
    }

m = folium.Map(location=[46.8566, 2.3522], zoom_start=6)

folium.GeoJson(
    zone,
    style_function=style_function,
    name='geojson'
).add_to(m)

colormap.add_to(m)

m


In [ ]:
dict_nb_ventes = pd.DataFrame(data["Code departement"].value_counts()).reset_index()
dict_nb_ventes.columns = ["Code departement", "Nombre de ventes"]
for i in dict_nb_ventes["Code departement"]:
    dict_nb_ventes.loc[dict_nb_ventes["Code departement"] == i, "Département"] = DEPARTMENTS.get(str(i))

act = '#fe9801'
temp = dict_nb_ventes
temp = temp.sort_values('Code departement', ascending=False)

fig = sp.make_subplots(rows=1, cols=2, subplot_titles=['Top 5 par départements les plus vendeurs', 'Top 5 par départements les moins vendeurs'])

fig.add_trace(go.Bar(x=temp.sort_values('Nombre de ventes', ascending=False).head(5).sort_values('Nombre de ventes', ascending=True)["Nombre de ventes"],
                     y=temp.sort_values('Nombre de ventes', ascending=False).head(5).sort_values('Nombre de ventes', ascending=True)["Département"],
                     text=temp.sort_values('Nombre de ventes', ascending=False).head(5).sort_values('Nombre de ventes', ascending=True)["Nombre de ventes"],
                     textposition='outside',
                     marker_color=act,
                     opacity=0.8,
                     orientation='h'),
              row=1, col=1)

fig.add_trace(go.Bar(x=temp.sort_values('Nombre de ventes', ascending=False).tail(5).sort_values('Nombre de ventes', ascending=False)["Nombre de ventes"],
                     y=temp.sort_values('Nombre de ventes', ascending=False).tail(5).sort_values('Nombre de ventes', ascending=False)["Département"],
                     text=temp.sort_values('Nombre de ventes', ascending=False).tail(5).sort_values('Nombre de ventes', ascending=False)["Nombre de ventes"],
                     textposition='outside',
                     marker_color=act,
                     opacity=0.8,
                     orientation='h'),
              row=1, col=2)

fig.update_layout(showlegend=False)

fig.update_xaxes(range=[50000, 110000], row=1, col=1)

fig.update_xaxes(range=[1000, 4000], row=1, col=2)

fig.show()

On remarque que le marché immobilier est scindé en plusieurs zones. l'Est de la France représente moins de vente que l'Ouest excepté le nord avec Lille. Les pôles les plus attractifs sont les grandes villes tel que Paris (qui influence égelement les département limitrophes), Lyon ou Lille et la côte avec une grande ville à proximité tel que Bordeaux dans la Gironde , Nantes situés en Loire atlantique, Montpellier dans l'Hérault et Rennes en Ile et-Villaine.

In [ ]:
temp = data[(data["Type local"] != "Dépendance")& (data["Type local"] != "Local industriel. commercial ou assimilé")].reset_index(drop = True)
temp = temp.groupby(['Code departement'])['Nombre pieces principales'].mean().reset_index()
temp.columns = ["Code departement","Nombre pieces principales"]
temp["Nombre pieces principales"] = round(temp["Nombre pieces principales"],1)
for i in temp["Code departement"]:
    temp.loc[temp["Code departement"] == i,"Département"] = DEPARTMENTS.get(str(i))
    
temp

temp = temp.sort_values('Code departement', ascending=False)

fig = px.bar(temp.sort_values('Nombre pieces principales', ascending=False).head(10).sort_values('Nombre pieces principales', ascending=True), 
             x="Nombre pieces principales", y="Code departement", text='Nombre pieces principales', orientation='h', 
             width=700, height=600, range_x = [0, 7], title='Nombre de piece moyen par departement')
fig.update_traces(marker_color=act, opacity=0.8, textposition='outside')
fig.show()

In [ ]:
type_counts = data['Nature mutation'].value_counts()

plot = type_counts.plot.pie(figsize=(10, 10),legend=True,colors=['olivedrab', 'rosybrown', 'gray', 'saddlebrown'])


In [ ]:
m_2 = data[(data["Type local"] != "Dépendance")& (data["Type local"] != "Local industriel. commercial ou assimilé")].reset_index(drop = True)

m_2["Prix mètre carré"] = m_2["Valeur fonciere"]/m_2["Surface reelle bati"]

property_changes = m_2.groupby('Code departement',as_index=False)['Prix mètre carré'].mean()
print(property_changes.sort_values("Prix mètre carré"))
property_changes.columns = ['Code', 'property_changes']

property_dict = property_changes.set_index('Code')['property_changes'].to_dict()

with open('../data/departements.geojson') as f:
    zone = json.load(f)

colormap = linear.YlOrRd_09.scale(
    0,
    property_changes.property_changes.max())

for feature in zone['features']:
    feature['properties']['property_changes'] = property_dict.get(feature['properties']['code'], None)

def style_function(feature):
    property_changes = feature['properties']['property_changes']
    return {
        'fillOpacity': 0.7,
        'weight': 2,
        'color': 'black',
        'fillColor': '#fff' if property_changes is None else colormap(property_changes)
    }

m = folium.Map(location=[46.8566, 2.3522], zoom_start=6)

folium.GeoJson(
    zone,
    style_function=style_function,
    name='geojson'
).add_to(m)

colormap.add_to(m)

m


### Pas analysable

In [ ]:
m2 = data[(data["Type local"] != "Dépendance")& (data["Type local"] != "Local industriel. commercial ou assimilé")].reset_index(drop = True)
m2["carrez_sum"] = m2["Surface Carrez du 1er lot"].fillna(0)  +  m2["Surface Carrez du 2eme lot"].fillna(0) + m2["Surface Carrez du 3eme lot"].fillna(0) + m2["Surface Carrez du 4eme lot"].fillna(0) + m2["Surface Carrez du 5eme lot"].fillna(0)
m2["Prix mètre carré"] = np.where(m2["carrez_sum"] != 0,m2["Valeur fonciere"]/m2["carrez_sum"],m2["Valeur fonciere"]/m2["Surface reelle bati"])
m2 = m2.drop(np.where(m2['Prix mètre carré'] > 25000)[0])
property_changes = m2.groupby('Code departement',as_index=False)['Prix mètre carré'].mean()

property_changes.columns = ['Code', 'property_changes']

property_dict = property_changes.set_index('Code')['property_changes'].to_dict()

with open('../data/departements.geojson') as f:
    zone = json.load(f)

colormap = linear.YlOrRd_09.scale(
    0,
    property_changes.property_changes.max())

for feature in zone['features']:
    feature['properties']['property_changes'] = property_dict.get(feature['properties']['code'], None)

def style_function(feature):
    property_changes = feature['properties']['property_changes']
    return {
        'fillOpacity': 0.7,
        'weight': 2,
        'color': 'black',
        'fillColor': '#fff' if property_changes is None else colormap(property_changes)
    }

m = folium.Map(location=[46.8566, 2.3522], zoom_start=6)

folium.GeoJson(
    zone,
    style_function=style_function,
    name='geojson'
).add_to(m)

colormap.add_to(m)

m


### Analysable

# Analyse foncière Inter-Annuel

Importation des librairies et des données ainsi que l'import des dapartements et des régions


In [ ]:
DEPARTMENTS = json.load(open("../data/departements_dict.json"))

REGIONS = json.load(open("../data/regions_dict.json"))

dth = '#ff2e63' 
rec = '#21bf73'
act = '#fe9801'
cnf = '#393e46'

data = pd.read_csv(r"../data/annee_traitee/2018.csv",low_memory=False,sep=",",header=0,index_col=0)
data = data.append(pd.read_csv(r"../data/annee_traitee/2019.csv",low_memory=False,sep=",",header=0,index_col=0))
data = data.append(pd.read_csv(r"../data/annee_traitee/2020.csv",low_memory=False,sep=",",header=0,index_col=0))
data = data.append(pd.read_csv(r"../data/annee_traitee/2021.csv",low_memory=False,sep=",",header=0,index_col=0))
data = data.append(pd.read_csv(r"../data/annee_traitee/2022.csv",low_memory=False,sep=",",header=0,index_col=0))

data["Date mutation"] = pd.to_datetime(data["Date mutation"], format="%Y-%m-%d")
data = data.reset_index(drop=True)

In [ ]:
display(data)

In [ ]:
temp = data[(data["Type local"] != "Dépendance")& (data["Type local"] != "Local industriel. commercial ou assimilé")].reset_index(drop = True)
temp["Month mutation"] = temp["Date mutation"].dt.month

temp['Prix mètre carré Paris 2018'] = temp[(temp["Code departement"] == '75')&(temp["Date mutation"].dt.year == 2018)]["Valeur fonciere"]/temp[(temp["Code departement"] == '75')&(temp["Date mutation"].dt.year == 2018)]["Surface reelle bati"]
temp['Prix mètre carré Paris 2019'] = temp[(temp["Code departement"] == '75')&(temp["Date mutation"].dt.year == 2019)]["Valeur fonciere"]/temp[(temp["Code departement"] == '75')&(temp["Date mutation"].dt.year == 2019)]["Surface reelle bati"]
temp['Prix mètre carré Paris 2022'] = temp[(temp["Code departement"] == '75')&(temp["Date mutation"].dt.year == 2022)]["Valeur fonciere"]/temp[(temp["Code departement"] == '75')&(temp["Date mutation"].dt.year == 2022)]["Surface reelle bati"]

temp =temp.replace(np.inf, np.nan)
display(temp['Prix mètre carré Paris 2018'].mean())
display(temp)

# temp['No. of Recovered to 1 Death Case'] = round(temp['Recovered']/temp['Deaths'], 3)
temp = temp.groupby('Month mutation')["Prix mètre carré Paris 2018","Prix mètre carré Paris 2019","Prix mètre carré Paris 2022"].mean().reset_index()

temp = temp.melt(id_vars='Month mutation', value_vars=['Prix mètre carré Paris 2018','Prix mètre carré Paris 2019','Prix mètre carré Paris 2022'], 
                 var_name='Departements', value_name='Value')

fig = px.line(temp, x="Month mutation", y="Value", color='Departements', log_y=True, 
              title='Evolution du prix du mètre carré à paris depuis 2018', color_discrete_sequence=[dth, rec,act])
fig.show()



In [ ]:
temp = data[(data["Type local"] != "Dépendance")& (data["Type local"] != "Local industriel. commercial ou assimilé")].reset_index(drop = True)
temp["Month mutation"] = temp["Date mutation"].dt.month
temp["carrez_sum"] = temp["Surface Carrez du 1er lot"].fillna(0)  +  temp["Surface Carrez du 2eme lot"].fillna(0) + temp["Surface Carrez du 3eme lot"].fillna(0) + temp["Surface Carrez du 4eme lot"].fillna(0) + temp["Surface Carrez du 5eme lot"].fillna(0)
temp["Prix mètre carré"] = np.where(temp["carrez_sum"] != 0,temp["Valeur fonciere"]/temp["carrez_sum"],temp["Valeur fonciere"]/temp["Surface reelle bati"])
temp = temp.drop(np.where(temp['Prix mètre carré'] > 25000)[0])

temp['Prix mètre carré Paris 2018'] = temp[(temp["Code departement"] == '75')&(temp["Date mutation"].dt.year == 2018)]["Prix mètre carré"]
temp['Prix mètre carré Paris 2019'] = temp[(temp["Code departement"] == '75')&(temp["Date mutation"].dt.year == 2019)]["Prix mètre carré"]
temp['Prix mètre carré Paris 2022'] = temp[(temp["Code departement"] == '75')&(temp["Date mutation"].dt.year == 2022)]["Prix mètre carré"]

temp =temp.replace(np.inf, np.nan)
display(temp['Prix mètre carré Paris 2018'].mean())
display(temp)

# temp['No. of Recovered to 1 Death Case'] = round(temp['Recovered']/temp['Deaths'], 3)
temp = temp.groupby('Month mutation')["Prix mètre carré Paris 2018","Prix mètre carré Paris 2019","Prix mètre carré Paris 2022"].mean().reset_index()

temp = temp.melt(id_vars='Month mutation', value_vars=['Prix mètre carré Paris 2018','Prix mètre carré Paris 2019','Prix mètre carré Paris 2022'], 
                 var_name='Departements', value_name='Value')

fig = px.line(temp, x="Month mutation", y="Value", color='Departements', log_y=True, 
              title='Evolution du prix du mètre carré à paris depuis 2018', color_discrete_sequence=[dth, rec,act])
fig.show()


In [ ]:
def location(row):
    if row['Code departement']=='75':
            return 'Paris'
    elif row['Code departement']=='6':
            return 'Alpes-Maritimes'
    elif row['Code departement']=='59':
        return 'Nord'
    else:
        return 'Reste de la France'


temp = data[(data["Type local"] != "Dépendance")& (data["Type local"] != "Local industriel. commercial ou assimilé")].reset_index(drop = True)
temp['Prix mètre carré'] = temp['Valeur fonciere']/temp['Surface reelle bati']
temp['Region'] = temp.apply(location, axis=1)
temp['Date'] = temp['Date mutation'].dt.strftime('%Y-%m-%d')
temp = temp.groupby(['Region', 'Date'])['Prix mètre carré'].mean().reset_index()
temp = temp.melt(id_vars=['Region', 'Date'], value_vars=['Prix mètre carré'], 
                 var_name='Case', value_name='Count').sort_values('Count')
# temp = temp.sort_values(['Date', 'Region', 'Case']).reset_index()
temp.head()

fig = px.bar(temp, y='Region', x='Count', color='Case', barmode='group', orientation='h',
             text='Count', title='Hubei - China - World', animation_frame='Date',
             color_discrete_sequence= [dth, rec, cnf], range_x=[0, 70000])
fig.update_traces(textposition='outside')
# fig.update_layout(uniformtext_minsize=8, uniformtext_mode='hide')
# fig.update_layout(yaxis={'categoryorder':'array', 
#                          'categoryarray':['Hubei','Other Chinese Provinces','Rest of the World']})
fig.show()

temp = data.copy()
temp = temp[(temp["Type local"] != "Dépendance")& (temp["Type local"] != "Local industriel. commercial ou assimilé")].reset_index(drop = True)
temp['Prix mètre carré'] = temp['Valeur fonciere']/temp['Surface reelle bati']
temp['Region'] = temp.apply(location, axis=1)
temp = temp.groupby('Region')['Prix mètre carré'].mean().reset_index()
temp = temp.melt(id_vars='Region', value_vars=['Prix mètre carré'], 
                 var_name='Case', value_name='Count').sort_values('Count')
temp.head()

fig = px.bar(temp, y='Region', x='Count', color='Case', barmode='group', orientation='h',
             text='Count', title='Paris - Nord - Alpes-Maritimes', 
             color_discrete_sequence= [dth, rec, cnf])
fig.update_traces(textposition='outside')
#fig.update_layout(uniformtext_minsize=8, uniformtext_mode='hide')
fig.show()

In [ ]:
carrez = data[(data["Type local"] != "Dépendance")& (data["Type local"] != "Local industriel. commercial ou assimilé")].reset_index(drop = True)
carrez["carrez_sum"] = data["Surface Carrez du 1er lot"].fillna(0)  +  data["Surface Carrez du 2eme lot"].fillna(0) + data["Surface Carrez du 3eme lot"].fillna(0) + data["Surface Carrez du 4eme lot"].fillna(0) + data["Surface Carrez du 5eme lot"].fillna(0)
carrez["Prix mètre carré"] = np.where(carrez["carrez_sum"] != 0,carrez["Valeur fonciere"]/carrez["carrez_sum"],carrez["Valeur fonciere"]/carrez["Surface reelle bati"])
carrez = carrez.drop(np.where(carrez['Prix mètre carré'] > 25000)[0])
display(carrez)


def location(row):
    if row['Code departement']=='75':
            return 'Paris'
    elif row['Code departement']=='6':
            return 'Alpes-Maritimes'
    elif row['Code departement']=='59':
        return 'Nord'
    else:
        return 'Reste de la France'


temp = carrez.sample(frac = 0.01)
temp['Region'] = temp.apply(location, axis=1)
temp['Date'] = temp['Date mutation'].dt.strftime('%Y-%m-%d')
temp = temp.groupby(['Region', 'Date'])['Prix mètre carré'].mean().reset_index()
temp = temp.melt(id_vars=['Region', 'Date'], value_vars=['Prix mètre carré'], 
                 var_name='Case', value_name='Count').sort_values('Count')
# temp = temp.sort_values(['Date', 'Region', 'Case']).reset_index()
temp.head()

fig = px.bar(temp, y='Region', x='Count', color='Case', barmode='group', orientation='h',
             text='Count', title='Hubei - China - World', animation_frame='Date',
             color_discrete_sequence= [dth, rec, cnf], range_x=[0, 15000])
fig.update_traces(textposition='outside')
# fig.update_layout(uniformtext_minsize=8, uniformtext_mode='hide')
# fig.update_layout(yaxis={'categoryorder':'array', 
#                          'categoryarray':['Hubei','Other Chinese Provinces','Rest of the World']})
fig.show()

temp = carrez.copy()
temp['Region'] = temp.apply(location, axis=1)
temp = temp.groupby('Region')['Prix mètre carré'].mean().reset_index()
temp = temp.melt(id_vars='Region', value_vars=['Prix mètre carré'], 
                 var_name='Case', value_name='Count').sort_values('Count')
temp.head()

fig = px.bar(temp, y='Region', x='Count', color='Case', barmode='group', orientation='h',
             text='Count', title='Paris - Nord - Alpes-Maritimes', 
             color_discrete_sequence= [cnf])
fig.update_traces(textposition='outside')
#fig.update_layout(uniformtext_minsize=8, uniformtext_mode='hide')
fig.show()